In [33]:
pip install PyPDF2 langchain langchain_groq langchain_community langchain_huggingface faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 24.6 MB/s eta 0:00:00


In [39]:
from PyPDF2 import PdfReader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

In [12]:
reader_attention = PdfReader("/content/NIPS-2017-attention-is-all-you-need-Paper.pdf")
number_of_pages = len(reader_attention.pages)
page_attention = reader_attention.pages[0]
text_attention = page_attention.extract_text()
text_attention

'Attention Is All You Need\nAshish Vaswani\x03\nGoogle Brain\navaswani@google.comNoam Shazeer\x03\nGoogle Brain\nnoam@google.comNiki Parmar\x03\nGoogle Research\nnikip@google.comJakob Uszkoreit\x03\nGoogle Research\nusz@google.com\nLlion Jones\x03\nGoogle Research\nllion@google.comAidan N. Gomez\x03y\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser\x03\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin\x03z\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable and requirin

In [13]:
reader_git = PdfReader("/content/git-cheat-sheet-education.pdf")
number_of_pages = len(reader_git.pages)
page_git = reader_git.pages[0]
text_git = page_git.extract_text()
text_git

"GIT CHEAT SHEET\nSTAGE & SNAPSHOT\nWorking with snapshots and the Git staging area\ngit status\nshow modiﬁed ﬁles in working directory, staged for your next commit\ngit add [file]\nadd a ﬁle as it looks now to your next commit (stage)\ngit reset [file]\nunstage a ﬁle while retaining the changes in working directory\ngit diff\ndiﬀ of what is changed but not staged\ngit diff --staged\ndiﬀ of what is staged but not yet committed\ngit commit -m “[descriptive message]”\ncommit your staged content as a new commit snapshotSETUP\nConﬁguring user information used across all local repositories\ngit config --global user.name “[firstname lastname]”\nset a name that is identiﬁable for credit when review version history\ngit config --global user.email “[valid-email]”\nset an email address that will be associated with each history marker\ngit config --global color.ui auto\nset automatic command line coloring for Git for easy reviewing\nSETUP & INIT\nConﬁguring user information, initializing and clon

In [14]:
text = text_attention + '\n' + text_git
text

"Attention Is All You Need\nAshish Vaswani\x03\nGoogle Brain\navaswani@google.comNoam Shazeer\x03\nGoogle Brain\nnoam@google.comNiki Parmar\x03\nGoogle Research\nnikip@google.comJakob Uszkoreit\x03\nGoogle Research\nusz@google.com\nLlion Jones\x03\nGoogle Research\nllion@google.comAidan N. Gomez\x03y\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser\x03\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin\x03z\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable and requirin

In [30]:
from os.path import split
splitted_text = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50).split_text(text)
splitted_text

['Attention Is All You Need\nAshish Vaswani\x03\nGoogle Brain\navaswani@google.comNoam Shazeer\x03\nGoogle Brain\nnoam@google.comNiki Parmar\x03\nGoogle Research\nnikip@google.comJakob Uszkoreit\x03\nGoogle Research\nusz@google.com\nLlion Jones\x03\nGoogle Research\nllion@google.comAidan N. Gomez\x03y\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser\x03\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin\x03z\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable and requiri

In [31]:
for i in range(len(splitted_text)):
  splitted_text[i] = Document(page_content=splitted_text[i])

In [28]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [34]:
vector_db = FAISS.from_documents(documents=splitted_text, embedding=embeddings)

In [42]:
retriever = vector_db.as_retriever()

In [38]:
llm = ChatGroq(api_key="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", model="gemma2-9b-it")

In [41]:
prompt = ChatPromptTemplate([
    ("system", "You are a helpful AI bot. your task is to generate response based on given user_input and relevant_context form vector search"),
    ("human", "{user_input}"),
    ("ai", "{relevant_context}")
])

In [50]:
chain = prompt | retriever | llm

In [45]:
chain.invoke({"user_input":"tell me about git"})

KeyError: "Input to ChatPromptTemplate is missing variables {'relevant_context'}.  Expected: ['relevant_context', 'user_input'] Received: ['user_input']\nNote: if you intended {relevant_context} to be part of the string and not a variable, please escape it with double curly braces like: '{{relevant_context}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

In [48]:
print(await retriever.ainvoke("about git"))

[Document(id='7df64a09-c62e-4b7d-8d48-ac055a151a34', metadata={}, page_content="git init\ninitialize an existing directory as a Git repository\ngit clone [url]\nretrieve an entire repository from a hosted location via URLBRANCH & MERGE\nIsolating work in branches, changing context, and integrating changes\ngit branch\nlist your branches. a * will appear next to the currently active branch\ngit branch [branch-name]\ncreate a new branch at the current commit\ngit checkout\nswitch to another branch and check it out into your working directory\ngit merge [branch]\nmerge the speciﬁed branch’s history into the current one\ngit log\nshow all commits in the current branch’s historyGit is the free and open source distributed version control system that's responsible for everything GitHub related that happens locally on your computer. This cheat sheet features the most important and commonly used Git commands for easy reference.INSTALLATION & GUIS\nWith platform speciﬁc installers for Git, GitHu